In [1]:
# packages
import numpy             as np 
import matplotlib.pyplot as plt
import pandas            as pd
import yaml, os, logging, sys, glob
pd.set_option("display.max_columns", None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from magicctapipe.scripts.lst1_magic import lst1_magic_create_irf, lst1_magic_dl2_to_dl3
from magicctapipe.scripts import create_dl3_index_files

# --- logging --- #
import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

# location of the scripts
sys.path.insert(0, '/fefs/aswg/workspace/juan.jimenez/stereo_analysis/scripts')
import auxiliar as aux
aux.params() # graphic parameters

########################################################
# source name
source_name = 'Mrk421'

# configuration file path
config_file   = f'/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/config{source_name}.yaml'
main_dir_irfs = f'/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs'
dir_dl3       = f'/fefs/aswg/workspace/juan.jimenez/data/dl3_{source_name}'

compute_irf = False
compute_dl3 = True
########################################################

In [2]:
dir_irfs = os.path.join(main_dir_irfs, f'irfs_{source_name}') 

with open(config_file, 'rb') as f:
    config = yaml.safe_load(f) 

    
# selecting the MC folder
if source_name == 'Crab':
    path_mcs = '/fefs/aswg/LST1MAGIC/mc/DL2/ST0316A/NSB1.5/GammaTest/v01.2/dec_2276/*.h5' # for CrabNebula
elif source_name == 'BLLac':
    path_mcs = '/fefs/aswg/LST1MAGIC/mc/DL2/ST0316A/NSB0.5/GammaTest/v01.2/dec_3476/*.h5' # for BLLac
elif source_name == 'Mrk421':
    path_mcs = '/fefs/aswg/LST1MAGIC/mc/DL2/ST0316A/NSB0.5/GammaTest/v01.2/dec_4822/*.h5' # for Mrk421
else:
    logger.error(f'The source introduced {source_name} have not specified RFs or is bad written.')
    
logger.info(f'Study of the source: {source_name}')
logger.info(f'\nAll data taken from the path: {path_mcs}')

Study of the source: Mrk421

All data taken from the path: /fefs/aswg/LST1MAGIC/mc/DL2/ST0316A/NSB0.5/GammaTest/v01.2/dec_4822/*.h5


# IRF

In [3]:
if compute_irf:
    # if you want to create it
    aux.createdir(dir_irfs)

    # extracting the MC data
    input_files_gamma = glob.glob(path_mcs)
    input_files_gamma.sort()
    logger.info(f'Found {len(input_files_gamma)} gamma MC files\n')

    for gamma_file in input_files_gamma:
        logger.info(f'\nComputing irf of {gamma_file}')
        lst1_magic_create_irf.create_irf(gamma_file, input_file_proton=None, input_file_electron=None, output_dir=dir_irfs, config=config)
        
        # reset the configuration reading
        with open(config_file, 'rb') as f:
            config = yaml.safe_load(f) 

    
logger.info(f'\n\nFinally the IRF folder will be {dir_irfs}')



Finally the IRF folder will be /fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421


## DL2 to DL3

In [4]:
aux.createdir(dir_dl3)

dir_dl2 = f'/fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_raw_dl2_{source_name}/*'
files_dl2 = glob.glob(dir_dl2)
files_dl2.sort()

runs_dl2 = sorted([int(f.split('/')[-1].split('.')[1].replace('Run', '')) for f in files_dl2])

logger.info(f'\nThe DL2 directory ---> {dir_dl2}\nWhere {len(files_dl2)} files are found')
logger.info(f'\nThe IRF directory --> {dir_irfs}')
logger.info(f'\nThe DL3 directory --> {dir_dl3}')

if compute_dl3 == True:
    for file, run in zip(files_dl2, runs_dl2):

        logger.info(f'\nComputing irf of {file}')
        lst1_magic_dl2_to_dl3.dl2_to_dl3(input_file_dl2=file, input_dir_irf=dir_irfs, output_dir=dir_dl3, config=config)

        # reset the configuration reading
        with open(config_file, 'rb') as f:
            config = yaml.safe_load(f) 
            
# We need to create index files
create_dl3_index_files(dir_dl3)


The DL2 directory ---> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_raw_dl2_Mrk421/*
Where 41 files are found

The IRF directory --> /fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421

The DL3 directory --> /fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421

Computing irf of /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_raw_dl2_Mrk421/dl2_LST-1_MAGIC.Run03217.h5

Input IRF directory: /fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421

The following IRF data files are found:
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_10.0deg_az_102.199deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_10.0deg_az_248.117deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_14.984deg_az_175.158deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_14.984deg_az_355.158deg_so

/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_68.284deg_az_30.445deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_68.284deg_az_329.555deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_70.732deg_az_29.341deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_70.732deg_az_330.659deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_73.142deg_az_28.021deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_73.142deg_az_331.979deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_75.226deg_az_318.974deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk42

/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_43.197deg_az_206.875deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_43.197deg_az_230.005deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_43.197deg_az_262.712deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_43.197deg_az_87.604deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_110.312deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_133.619deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_152.343deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk


Extra header:
    TELESCOP: CTA-N
    INSTRUME: LST-1_MAGIC
    FOVALIGN: RADEC
    QUAL_CUT: disp_diff_mean < 0.22
    EVT_TYPE: software
    DL2_WEIG: variance
    GH_EFF: 0.9
    GH_MIN: 0.05
    GH_MAX: 0.85
    RAD_MAX: 0.2 

Input DL2 data file: /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_raw_dl2_Mrk421/dl2_LST-1_MAGIC.Run03218.h5

In total 8095 stereo events are found:
    M1_M2 (type 0): 1484 events (18.3%)
    LST1_M1 (type 1): 798 events (9.9%)
    LST1_M2 (type 2): 1645 events (20.3%)
    LST1_M1_M2 (type 3): 4168 events (51.5%) 

Extracting the events of the 'software' type...
--> 6611 stereo events

Calculating the dead time correction factor...
LST-1: 0.96
MAGIC(-II): 0.992
--> Total correction factor: 0.952

Target point in (cos(Zd), Az): [0.69837, 1.09676]

Interpolation method: nearest

Interpolating the effective area...
Interpolating the energy dispersion...
Interpolating the dynamic gammaness cuts...

Gammaness cut table:

        low                  high   

/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_60.528deg_az_251.19deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_60.528deg_az_99.126deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_60.66deg_az_32.517deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_60.66deg_az_327.483deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_63.255deg_az_32.039deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_63.255deg_az_327.961deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_65.976deg_az_31.344deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/ir

...

/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_75.522deg_az_26.462deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_75.522deg_az_333.538deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz

Grid points in (cos(Zd), Az):
array(  0.98480775 1.78370904 
        0.98480775 4.33045858 
        0.96599806 3.05708381 
        0.96599806 6.19867646 
        0.91615298 1.75855885 
        0.91615298 4.52502789 
        0.84750197 1.7840232  
        0.84750197 2.37457281 
        0.84750197 3.05708381 
        0.84750197 3.73959482 
        0.84750197 4.33014442 
        0.84750197 6.19867646 
        0.8100007  0.30476939 
        0.8100007  5.97841591 
        0.77000124 0.42469097 
        0.77000124 5.85849434 
        0.72900447 2.09982308 
        0.72900447 2.50351773 
        0.72900447 3.05708381 
        0.72900447 3.61064989 
        0.72900447 4.01434455 
        0.72900447 4.58518938 
        0.72900447

/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_175.158deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_197.973deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_216.698deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_240.004deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_301.217deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.374deg_az_49.119deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_52.41deg_az_32.381deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk42

--> 8920 stereo events

Calculating the dead time correction factor...
LST-1: 0.951
MAGIC(-II): 0.994
--> Total correction factor: 0.946

Target point in (cos(Zd), Az): [0.91635, 5.25658]

Interpolation method: nearest

Interpolating the effective area...
Interpolating the energy dispersion...
Interpolating the dynamic gammaness cuts...

Gammaness cut table:

        low                  high                 cut        
        TeV                  TeV                             
-------------------- -------------------- -------------------
                0.01 0.015848931924611134                0.05
0.015848931924611134 0.025118864315095794                0.05
0.025118864315095794 0.039810717055349734                0.05
0.039810717055349734  0.06309573444801933                0.05
 0.06309573444801933                  0.1                0.05
                 0.1  0.15848931924611143  0.5885525453098017
                 ...                  ...                 ...
  39.8107170553497

/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_63.255deg_az_327.961deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_65.976deg_az_31.344deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_65.976deg_az_328.656deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_68.068deg_az_119.073deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_68.068deg_az_141.683deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_68.068deg_az_175.158deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk421/irf_zd_68.068deg_az_208.633deg_software_gh_dyn0.9_theta_glob0.2deg.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl2/irfs/irfs_Mrk

/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run04649.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run04939.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run04962.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run07043.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run07182.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run07183.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run07184.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run07290.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run07323.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run07370.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run08116.fits.gz
/fefs/aswg/workspace/juan.jimenez/data/dl3_Mrk421/dl3_LST-1_MAGIC.Run08117.fits.gz
/fef